In [1]:
import torch
from transformers.models.opt.modeling_opt import OPTAttention, OPTDecoderLayer, OPTForCausalLM
from transformers import GPT2Tokenizer

/mnt/lustre/GPU8/home/liutielong/anaconda3/envs/OmniQuant/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def calculate(time_calc, time_read, runs=512, simu_time1=30, simu_time2=100, simu_time3=180):
    time_total = time_calc + time_read
    time_calc_avg = "{:.3e}".format(time_calc / runs)
    time_read_avg = "{:.3e}".format(time_read / runs)
    time_total_avg = time_total / runs
    acc1 = simu_time1 / time_total_avg
    acc2 = simu_time2 / time_total_avg
    acc3 = simu_time3 / time_total_avg
    acc_avg = pow(acc1 * acc2 * acc3, 1/3)
    print(f"计算平均用时: {time_calc_avg}")
    print(f"读取平均用时: {time_read_avg}")
    print(f"总用时: {time_total}")
    print(f"平均总用时: {time_total_avg}")
    print(f"加速倍数1: {acc1}")
    print(f"加速倍数2: {acc2}")
    print(f"加速倍数3: {acc3}")
    print(f"几何平均加速倍数:{acc_avg}")

time_calc = 4.380
time_read = 3.777
runs = 512
simu_time1 = 30
simu_time2 = 100
simu_time3 = 180
calculate(time_calc, time_read, runs, simu_time1, simu_time2, simu_time3)

计算平均用时: 8.555e-03
读取平均用时: 7.377e-03
总用时: 8.157
平均总用时: 0.015931640625
加速倍数1: 1883.045237219566
加速倍数2: 6276.817457398553
加速倍数3: 11298.271423317396
几何平均加速倍数:5111.371164753922


In [2]:
import torch
tensor = torch.tensor([1, 5, 3, 7, 2, 8, 4])

# 计算张量的最大值和最小值
max_value = torch.max(tensor)
min_value = torch.min(tensor)


In [13]:
x_int = torch.arange(1,65)
x_int = torch.log(x_int) / torch.log(torch.tensor(2))
x_int.round_() - 1 
x_int = 2 ** x_int
print(x_int[1:])



tensor([ 2.,  4.,  4.,  4.,  8.,  8.,  8.,  8.,  8.,  8., 16., 16., 16., 16.,
        16., 16., 16., 16., 16., 16., 16., 32., 32., 32., 32., 32., 32., 32.,
        32., 32., 32., 32., 32., 32., 32., 32., 32., 32., 32., 32., 32., 32.,
        32., 32., 64., 64., 64., 64., 64., 64., 64., 64., 64., 64., 64., 64.,
        64., 64., 64., 64., 64., 64., 64.])


In [82]:
ori_int = torch.arange(0, 64)
print(ori_int)
ori_int = torch.log(ori_int) / torch.log(torch.tensor(2))
ori_int.round_()
ori_int = (2 ** ori_int).to(dtype=int)
print(ori_int)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63])
tensor([ 0,  1,  2,  4,  4,  4,  8,  8,  8,  8,  8,  8, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64])


In [80]:
a = torch.log(torch.tensor(0))
2 ** (a)

tensor(0.)

In [72]:
import numpy


class LogQuant2:
    def __init__(self,layer,bitwidth=3):
        self.layer_data = layer
        self.width = bitwidth
        self.maxima = numpy.amax(layer)
        self.minima = numpy.amin(layer)
        self.scale = (0.95 * self.maxima - 0.92 * self.minima) / 2
        self.sign = numpy.sign(layer)


    def __clip(self, x):
        if(x < 1 - 2**self.width):
            return 2
        elif(x > 0):
            return 0
        else:
            return x

    def __round(self,x):
        halfprecision = (2**(numpy.ceil(x)) - 2**(numpy.floor(x)))/2
        fractional = 2**(x) - 2**(numpy.floor(x))
        if fractional >= halfprecision:
            return numpy.ceil(x)
        else:
            return numpy.floor(x)

    @property
    def log_quantized(self):
        round = numpy.vectorize(self.__round)
        clip = numpy.vectorize(self.__clip)
        # numpy.log2(0) -> -infinity == float("-inf") which will be used in clip method
        return numpy.array(clip(round(numpy.log2(numpy.abs(self.layer_data / self.scale)))),dtype=numpy.int8)

    @property
    def anti_quantized(self):
        x = numpy.power(2.0, self.log_quantized)
        x = numpy.array(x,dtype=numpy.float32)
        x[abs(x) == numpy.power(2.0, 2)] = 0
        return x * self.sign * self.scale

numpy.random.seed(10)
layer = numpy.random.randn(20, 20)


In [73]:
print(layer)

[[ 1.3315865   0.71527897 -1.54540029 -0.00838385  0.62133597 -0.72008556
   0.26551159  0.10854853  0.00429143 -0.17460021  0.43302619  1.20303737
  -0.96506567  1.02827408  0.22863013  0.44513761 -1.13660221  0.13513688
   1.484537   -1.07980489]
 [-1.97772828 -1.7433723   0.26607016  2.38496733  1.12369125  1.67262221
   0.09914922  1.39799638 -0.27124799  0.61320418 -0.26731719 -0.54930901
   0.1327083  -0.47614201  1.30847308  0.19501328  0.40020999 -0.33763234
   1.25647226 -0.7319695 ]
 [ 0.66023155 -0.35087189 -0.93943336 -0.48933722 -0.80459114 -0.21269764
  -0.33914025  0.31216994  0.56515267 -0.14742026 -0.02590534  0.2890942
  -0.53987907  0.70816002  0.84222474  0.2035808   2.39470366  0.91745894
  -0.11227247 -0.36218045]
 [-0.23218226 -0.5017289   1.12878515 -0.69781003 -0.08112218 -0.52929608
   1.04618286 -1.41855603 -0.36249918 -0.12190569  0.31935642  0.4609029
  -0.21578989  0.98907246  0.31475378  2.46765106 -1.50832149  0.62060066
  -1.04513254 -0.79800882]
 [ 1.9

In [74]:
loglayer = LogQuant2(layer,2)

In [75]:
layer_quant = loglayer.log_quantized
print(layer_quant)

[[-1 -2 -1  2 -2 -2 -3  2  2  2 -3 -1 -2 -1  2 -3 -1  2 -1 -1]
 [-1 -1 -3  0 -1 -1  2 -1 -3 -2 -3 -2  2 -3 -1  2 -3 -3 -1 -2]
 [-2 -3 -2 -3 -2  2 -3 -3 -2  2  2 -3 -2 -2 -2  2  0 -2  2 -3]
 [ 2 -2 -1 -2  2 -2 -1 -1 -3  2 -3 -3  2 -2 -3  0 -1 -2 -1 -2]
 [ 0 -1 -1  2  2  0  2 -3  2  0 -1  2  2 -2 -2 -2 -3  2 -2 -1]
 [ 2 -1 -2 -3  2 -3 -3 -2 -2 -2 -3 -3  0 -2 -3 -3 -3  2 -3 -1]
 [-2 -1 -1  0 -1 -3 -1  2 -1 -2  2 -1 -2 -3 -1  2 -1  2  0 -1]
 [-3 -1  2 -3 -3 -1 -1 -3 -3 -1 -2 -3 -1 -3  2  2 -2 -2 -2  0]
 [-1  2 -3 -2  0 -1 -3 -2 -3 -2 -1 -2 -1 -2 -2 -3 -2 -2 -1 -3]
 [-1 -2 -2 -2  0  2  2  2 -3 -2 -3  2 -1  2 -3 -1 -2 -1 -1 -2]
 [ 2 -1 -1  2 -1 -1 -2  2 -2 -1 -3 -1 -3 -3 -2  2 -1 -1 -2 -1]
 [-2 -2 -2 -1  2 -3 -2 -3 -3  2 -1 -2  2 -3  2 -3 -3 -2 -2 -1]
 [ 0 -2 -1 -1 -1 -3 -2 -1 -1 -1  2 -3 -2 -2  2 -1 -2 -2  2 -2]
 [-1 -3 -2 -1 -2  2 -2 -2 -3 -2  2  2  2 -1 -2  2 -1 -2 -1  2]
 [-1 -3 -2 -3  0 -3 -2  2 -2  0 -2 -3 -2  2 -1 -3  2  2  2 -2]
 [-2 -2  2 -2 -3 -3 -1  2 -3 -1  0 -1 -1 -2 -2 -2  2  2

In [76]:
layer_dequant = loglayer.anti_quantized
print(layer_dequant)

[[ 1.32178596  0.66089298 -1.32178596 -0.          0.66089298 -0.66089298
   0.33044649  0.          0.         -0.          0.33044649  1.32178596
  -0.66089298  1.32178596  0.          0.33044649 -1.32178596  0.
   1.32178596 -1.32178596]
 [-1.32178596 -1.32178596  0.33044649  2.64357191  1.32178596  1.32178596
   0.          1.32178596 -0.33044649  0.66089298 -0.33044649 -0.66089298
   0.         -0.33044649  1.32178596  0.          0.33044649 -0.33044649
   1.32178596 -0.66089298]
 [ 0.66089298 -0.33044649 -0.66089298 -0.33044649 -0.66089298 -0.
  -0.33044649  0.33044649  0.66089298 -0.         -0.          0.33044649
  -0.66089298  0.66089298  0.66089298  0.          2.64357191  0.66089298
  -0.         -0.33044649]
 [-0.         -0.66089298  1.32178596 -0.66089298 -0.         -0.66089298
   1.32178596 -1.32178596 -0.33044649 -0.          0.33044649  0.33044649
  -0.          0.66089298  0.33044649  2.64357191 -1.32178596  0.66089298
  -1.32178596 -0.66089298]
 [ 2.64357191  1.321

In [86]:
class Quantizer():
    def __init__(self, symmetric, n_bits) -> None:
        self.n_bits = n_bits
        if symmetric:
            self.qmin = -2 ** (2 ** (n_bits - 1) - 1)
            self.qmax = 2 ** (2 ** (n_bits - 1) - 2)
        else:
            self.qmin = 0
            self.qmax = 2 ** (2 ** n_bits -2)

q1 = Quantizer(symmetric=False, n_bits=3)
print(q1.qmax)
print(q1.qmin)

64
0
